In [1]:
### REAL POC CELL ##

"""
Created on Mon Feb 10 16:36:55 2020
No stealy 
@author: ecouv
"""
# Qt5 related stuff
from PyQt5.QtWidgets import QApplication, QMainWindow, QMenu, QVBoxLayout, QSizePolicy, QMessageBox, QWidget, QPushButton
from PyQt5.QtGui import QIcon

from PyQt5 import QtWidgets

import sys
sys.path.insert(0, 'UserInterface/')
from GUI import Ui_TranscriptEditor
import Render


# transcription library stuff
from google.cloud import speech_v1
from scipy.io import wavfile
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg') # for QT5

# stuff
import math
import numpy as np
import time
import speech_recognition as sr
import os
from pathlib import Path
import copy

fp= 'RawAudio/case1.wav'
fpp='RawAudio/case2.wav'


# translate API transcript to one easier to use
print(1)
transcript1 = Render.Transcript()
transcript1.ibm_recog('case1.wav',fp,'audio/wav')
print(2)
transcript2 = Render.Transcript()
transcript2.ibm_recog('Ana Marie Cox.wav',fpp,'audio/wav')

#print(transcript1.sr, transcript2.sr)

# GUI TESTING
print('Launching UI')
qApp = QtWidgets.QApplication(sys.argv)

# aw = ApplicationWindow, currently using TranscriptEditor from qtdesign
aw = Ui_TranscriptEditor()
aw.setupUi(aw)
aw.setupUiManual()

tarray = [transcript1, transcript2]
#tarray = [transcript1]
aw.launchInit(tarray, len(tarray))

aw.show()
sys.exit(qApp.exec_())



1
Transcript Recognizes Mono audio
2
Transcript Recognizes Mono audio
Launching UI
shift applied
0
right 3 33280 353280
Rendered, check next tab


Rendered Sound,Your browser does not support the audio element.


shift applied
0
0


ValueError: could not broadcast input array from shape (0) into shape (18560)

SystemExit: 0

C:\Users\ecouv\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
